<a href="https://colab.research.google.com/github/aekanun2020/AdvancedStat/blob/main/prove_one_neuronRNN_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.nn import Parameter
from torch.autograd import Variable
import torch.nn.functional as F
import math
import numpy as np

# กำหนด seed เพื่อให้ผลลัพธ์คงที่
torch.manual_seed(9)

# ==================== ส่วนที่ 1: การเตรียมโมเดลและข้อมูล ====================
print("=" * 60)
print("ส่วนที่ 1: การเตรียมโมเดลและข้อมูล")
print("=" * 60)

# สร้าง RNN Cell โดยมี input_size=1, hidden_size=1, และไม่มี bias
rnn_model = torch.nn.RNNCell(input_size=1, hidden_size=1, bias=False)

# Linear layer สำหรับแปลง hidden state เป็น output
output_layer = torch.nn.Linear(1, 1, bias=False)

# แสดงค่า parameters ทั้งหมดใน RNN Cell พร้อมคำอธิบาย
weights = list(rnn_model.parameters())
w_ih = weights[0]  # Input-to-Hidden (Weight W)
w_hh = weights[1]  # Hidden-to-Hidden (Weight U)
print(f"Parameters เริ่มต้นของ RNN Cell:")
print(f"Weight W (input-to-hidden): {w_ih.item():.4f}")
print(f"Weight U (hidden-to-hidden): {w_hh.item():.4f}")

# กำหนดค่า V
with torch.no_grad():
    output_layer.weight.copy_(torch.tensor([[0.5]]))
v_weight = output_layer.weight.item()
print(f"Weight V (hidden-to-output): {v_weight:.4f}")

# สร้างข้อมูลสำหรับ train: sine wave
seq_length = 20
time_steps = torch.linspace(0, 4*math.pi, seq_length)
data = torch.sin(time_steps).unsqueeze(1)

# แบ่งเป็น input และ target (เราต้องการทำนาย data[t+1] จาก data[t])
x_train = data[:-1]  # ทุกค่ายกเว้นค่าสุดท้าย
y_train = data[1:]   # ทุกค่ายกเว้นค่าแรก

print(f"\nสร้างชุดข้อมูล sine wave ความยาว {seq_length} ค่า")
print(f"จำนวน time steps ในชุดข้อมูล: {len(x_train)}")
print(f"Shape ของ input (x): {x_train.shape}, target (y): {y_train.shape}")

print("\nข้อมูล input และ target ทั้งหมด:")
print("  Time Step |    Input (x)    |   Target (y)   ")
print("-" * 50)
for i in range(len(x_train)):
    print(f"     {i+1:2d}     |    {x_train[i].item():+.6f}    |    {y_train[i].item():+.6f}    ")

# ==================== ส่วนที่ 2: การคำนวณก่อนการเทรน ====================
print("\n" + "=" * 60)
print("ส่วนที่ 2: การคำนวณก่อนการเทรน (Forward Pass แบบละเอียด)")
print("=" * 60)

# คำนวณ forward pass ก่อนการเทรน
h = torch.zeros(1, 1)  # เริ่มต้นด้วย hidden state = 0

print("\nการคำนวณแบบละเอียดในแต่ละ time step ก่อนการเทรน:")
print("-" * 90)
print("Time Step |   Input (x)  |    h_{t-1}    |       W·x       |       U·h       |     h_t = tanh(W·x + U·h)     |      y_t = sigmoid(V·h_t)     |   Target   |    Error   ")
print("-" * 90)

total_squared_error = 0
predictions = []

for t in range(len(x_train)):
    # รับค่า input และค่า target
    x_t = x_train[t].unsqueeze(0)
    y_target = y_train[t].item()

    # คำนวณ forward pass ของ RNN Cell
    Wx = w_ih.item() * x_t.item()
    Uh = w_hh.item() * h.item()
    next_h = torch.tanh(rnn_model.weight_ih * x_t + rnn_model.weight_hh * h)
    y_pred = torch.sigmoid(output_layer(next_h))

    # คำนวณ error
    error = y_pred.item() - y_target
    squared_error = error ** 2
    total_squared_error += squared_error

    # เก็บผลการทำนาย
    predictions.append(y_pred.item())

    # แสดงข้อมูลการคำนวณ
    print(f"    {t+1:2d}    | {x_t.item():+.6f} | {h.item():+.6f} | {Wx:+.6f} | {Uh:+.6f} | {next_h.item():+.6f} | {y_pred.item():+.6f} | {y_target:+.6f} | {error:+.6f}")

    # อัพเดท hidden state สำหรับ time step ถัดไป
    h = next_h

# คำนวณ MSE
mse = total_squared_error / len(x_train)
print("-" * 90)
print(f"Mean Squared Error (MSE) ก่อนการเทรน: {mse:.6f}")

# ==================== ส่วนที่ 3: การเทรนโมเดล ====================
print("\n" + "=" * 60)
print("ส่วนที่ 3: การเทรนโมเดล RNN")
print("=" * 60)

# สร้างโมเดลใหม่เพื่อการเทรน (ใช้ค่าเริ่มต้นเดียวกับโมเดลก่อนหน้า)
torch.manual_seed(9)
rnn_train = torch.nn.RNNCell(input_size=1, hidden_size=1, bias=False)
output_train = torch.nn.Linear(1, 1, bias=False)

# กำหนดค่าเริ่มต้นให้เหมือนกับโมเดลก่อนหน้า
with torch.no_grad():
    rnn_train.weight_ih.copy_(w_ih)
    rnn_train.weight_hh.copy_(w_hh)
    output_train.weight.copy_(torch.tensor([[0.5]]))

# สร้าง optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(list(rnn_train.parameters()) + list(output_train.parameters()), lr=learning_rate)
print(f"Optimizer: SGD with learning rate = {learning_rate}")
print("Loss function: Mean Squared Error (MSE)")

# จำนวน epochs
num_epochs = 10
print(f"การเทรนจะใช้ {num_epochs} epochs กับข้อมูล {len(x_train)} time steps")

# เตรียมตัวแปรสำหรับเก็บประวัติ
losses = []
w_history = [rnn_train.weight_ih.item()]
u_history = [rnn_train.weight_hh.item()]
v_history = [output_train.weight.item()]

# Train โมเดล
print("\nเริ่มการเทรน...")
for epoch in range(num_epochs):
    # เริ่มต้นจาก hidden state เป็น 0
    h = torch.zeros(1, 1)
    h.requires_grad = True  # ต้องเก็บ gradient เพื่อการเทรน

    # เก็บค่าทำนายทั้งหมด
    predictions = []

    # Forward pass ผ่านทุก time steps
    for t in range(len(x_train)):
        h = rnn_train(x_train[t].unsqueeze(0), h)
        y_pred = output_train(h)
        predictions.append(y_pred)

    # รวมการทำนายทั้งหมดเป็น tensor เดียว
    predictions = torch.cat(predictions)

    # คำนวณ loss
    loss = F.mse_loss(predictions, y_train)
    losses.append(loss.item())

    # Backward pass และการปรับค่า parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # เก็บค่า parameters หลังการปรับปรุง
    w_history.append(rnn_train.weight_ih.item())
    u_history.append(rnn_train.weight_hh.item())
    v_history.append(output_train.weight.item())

    # แสดงความคืบหน้า
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")
    print(f"  W: {rnn_train.weight_ih.item():.6f}, U: {rnn_train.weight_hh.item():.6f}, V: {output_train.weight.item():.6f}")

# แสดงการเปลี่ยนแปลงของ parameters
print("\nการเปลี่ยนแปลงของ parameters ในแต่ละ epoch:")
print("  Epoch |      W       |      U       |      V       |    Loss    ")
print("-" * 60)
for i in range(num_epochs + 1):  # +1 เพราะรวมค่าเริ่มต้นก่อน epoch 1
    if i == 0:
        epoch_label = "Initial"
    else:
        epoch_label = f"{i:5d}"

    loss_value = losses[i-1] if i > 0 else mse  # ใช้ MSE ก่อนเทรนสำหรับค่าเริ่มต้น
    print(f"  {epoch_label} | {w_history[i]:+.6f} | {u_history[i]:+.6f} | {v_history[i]:+.6f} | {loss_value:.6f}")

# ==================== ส่วนที่ 4: การคำนวณหลังการเทรน ====================
print("\n" + "=" * 60)
print("ส่วนที่ 4: การคำนวณหลังการเทรน (Forward Pass แบบละเอียด)")
print("=" * 60)

# คำนวณ forward pass หลังการเทรน
h = torch.zeros(1, 1)  # เริ่มต้นด้วย hidden state = 0

print("\nการคำนวณแบบละเอียดในแต่ละ time step หลังการเทรน:")
print("-" * 90)
print("Time Step |   Input (x)  |    h_{t-1}    |       W·x       |       U·h       |     h_t = tanh(W·x + U·h)     |      y_t = sigmoid(V·h_t)     |   Target   |    Error   ")
print("-" * 90)

total_squared_error = 0
final_predictions = []

with torch.no_grad():  # ไม่ต้องคำนวณ gradient ในการทดสอบ
    for t in range(len(x_train)):
        # รับค่า input และค่า target
        x_t = x_train[t].unsqueeze(0)
        y_target = y_train[t].item()

        # คำนวณ forward pass ของ RNN Cell
        Wx = rnn_train.weight_ih.item() * x_t.item()
        Uh = rnn_train.weight_hh.item() * h.item()
        next_h = torch.tanh(rnn_train.weight_ih * x_t + rnn_train.weight_hh * h)
        y_pred = torch.sigmoid(output_train(next_h))

        # คำนวณ error
        error = y_pred.item() - y_target
        squared_error = error ** 2
        total_squared_error += squared_error

        # เก็บผลการทำนาย
        final_predictions.append(y_pred.item())

        # แสดงข้อมูลการคำนวณ
        print(f"    {t+1:2d}    | {x_t.item():+.6f} | {h.item():+.6f} | {Wx:+.6f} | {Uh:+.6f} | {next_h.item():+.6f} | {y_pred.item():+.6f} | {y_target:+.6f} | {error:+.6f}")

        # อัพเดท hidden state สำหรับ time step ถัดไป
        h = next_h

# คำนวณ MSE
final_mse = total_squared_error / len(x_train)
print("-" * 90)
print(f"Mean Squared Error (MSE) หลังการเทรน: {final_mse:.6f}")
print(f"เปรียบเทียบกับ MSE ก่อนการเทรน: {mse:.6f}")
improvement = ((mse - final_mse) / mse) * 100
print(f"การเทรนช่วยลด MSE ลง: {improvement:.2f}%")

# ==================== ส่วนที่ 5: สรุปการเปลี่ยนแปลงของ Parameters ====================
print("\n" + "=" * 60)
print("ส่วนที่ 5: สรุปการเปลี่ยนแปลงของ Parameters")
print("=" * 60)

print("Parameters ก่อนการเทรน:")
print(f"  W: {w_ih.item():.6f}")
print(f"  U: {w_hh.item():.6f}")
print(f"  V: {v_weight:.6f}")

print("\nParameters หลังการเทรน:")
print(f"  W: {rnn_train.weight_ih.item():.6f}")
print(f"  U: {rnn_train.weight_hh.item():.6f}")
print(f"  V: {output_train.weight.item():.6f}")

w_change = ((rnn_train.weight_ih.item() - w_ih.item()) / w_ih.item()) * 100
u_change = ((rnn_train.weight_hh.item() - w_hh.item()) / w_hh.item()) * 100
v_change = ((output_train.weight.item() - v_weight) / v_weight) * 100

print("\nการเปลี่ยนแปลงของ parameters (%):")
print(f"  W: {w_change:+.2f}%")
print(f"  U: {u_change:+.2f}%")
print(f"  V: {v_change:+.2f}%")

# ==================== ส่วนที่ 6: บทบาทของ y ในการเทรน ====================
print("\n" + "=" * 60)
print("ส่วนที่ 6: บทบาทของ y ในการเทรน")
print("=" * 60)

print("1. การคำนวณค่า y ในแต่ละ time step:")
print("   - ในแต่ละ time step, RNN คำนวณ hidden state (h_t) และ output (y_t)")
print("   - y_t = sigmoid(V·h_t) เป็นผลลัพธ์ที่โมเดลใช้ในการทำนาย")
print("   - ในตัวอย่างนี้ เรามี 19 time steps จึงมีการคำนวณ y ทั้งหมด 19 ค่าในแต่ละ epoch")

print("\n2. บทบาทของ y ในการคำนวณ loss:")
print("   - ค่า y ในแต่ละ time step ถูกนำไปเปรียบเทียบกับค่าเป้าหมาย (target)")
print("   - ความแตกต่างระหว่าง y กับ target คือ error ซึ่งใช้ในการคำนวณ loss")
print("   - loss = MSE = (1/n) * Σ(y - target)²")
print("   - จะเห็นว่า loss ลดลงจาก {:.6f} เป็น {:.6f} หลังการเทรน".format(mse, final_mse))

print("\n3. บทบาทของ y ในการคำนวณ gradient และปรับปรุง parameters:")
print("   - ค่า gradient ของ loss เทียบกับ y (∂loss/∂y) ถูกคำนวณในขั้นตอน backward")
print("   - จากนั้น gradient จะถูกส่งผ่านไปยัง parameters (W, U, V) ตามกฎลูกโซ่")
print("   - ในแต่ละ epoch, parameters ถูกปรับด้วย gradient และ learning rate")
print("   - การปรับ parameters ทำให้ y ใกล้เคียงกับ target มากขึ้นเรื่อยๆ")

print("\n4. ความสัมพันธ์ระหว่าง h กับ y:")
print("   - hidden state (h) เก็บข้อมูลจากทั้ง input ปัจจุบันและข้อมูลในอดีต")
print("   - output (y) แปลงข้อมูลจาก hidden state ให้อยู่ในรูปแบบที่เหมาะสมกับงาน")
print("   - ในตัวอย่างนี้ sigmoid ถูกใช้เพื่อแปลง h เป็น y ที่อยู่ในช่วง (0, 1)")
print("   - การปรับปรุง W, U จะส่งผลต่อ h และในที่สุดส่งผลต่อ y")
print("   - การปรับปรุง V จะส่งผลโดยตรงต่อการแปลง h เป็น y")

print("\n5. สรุปความสำคัญของ y:")
print("   - y เป็นตัวเชื่อมระหว่างโมเดลกับข้อมูลจริง")
print("   - y ใช้ในการประเมินประสิทธิภาพของโมเดลผ่าน loss function")
print("   - y ใช้ในการปรับปรุงโมเดลผ่านกระบวนการ backpropagation")
print("   - โดยสรุป: ถ้าไม่มี y เราจะไม่สามารถเทรนโมเดล RNN ได้เลย")

ส่วนที่ 1: การเตรียมโมเดลและข้อมูล
Parameters เริ่มต้นของ RNN Cell:
Weight W (input-to-hidden): 0.3116
Weight U (hidden-to-hidden): -0.3960
Weight V (hidden-to-output): 0.5000

สร้างชุดข้อมูล sine wave ความยาว 20 ค่า
จำนวน time steps ในชุดข้อมูล: 19
Shape ของ input (x): torch.Size([19, 1]), target (y): torch.Size([19, 1])

ข้อมูล input และ target ทั้งหมด:
  Time Step |    Input (x)    |   Target (y)   
--------------------------------------------------
      1     |    +0.000000    |    +0.614213    
      2     |    +0.614213    |    +0.969400    
      3     |    +0.969400    |    +0.915773    
      4     |    +0.915773    |    +0.475947    
      5     |    +0.475947    |    -0.164595    
      6     |    -0.164595    |    -0.735724    
      7     |    -0.735724    |    -0.996584    
      8     |    -0.996584    |    -0.837166    
      9     |    -0.837166    |    -0.324700    
     10     |    -0.324700    |    +0.324700    
     11     |    +0.324700    |    +0.837167    
    